# Battle of the Neighbourhoods
## Identifying a suitable location in Toronto to open a new Yoga Studio

### Import Dependencies

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Foursquare Credentials and Version

In [201]:
CLIENT_ID = 'EFE5MSWQNT0Y3F5EEU2ZB2YWSYINQQMTF5ITISJ34PI2OOU4' # your Foursquare ID
CLIENT_SECRET = 'CXOODZSKSF0JETLUQ43ZSXEZ1TAXMSAB0S424NPOWOPWVF4X' # your Foursquare Secret
VERSION = '20191008'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EFE5MSWQNT0Y3F5EEU2ZB2YWSYINQQMTF5ITISJ34PI2OOU4
CLIENT_SECRET:CXOODZSKSF0JETLUQ43ZSXEZ1TAXMSAB0S424NPOWOPWVF4X


### New York data

#### Use wget command to import

In [175]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

#### Define new variable using 'features' key

In [176]:
neighborhoods_data = newyork_data['features']

#### Convert to pandas dataframe

In [177]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
 # define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)   

neighborhoods


,Borough,Neighborhood,Latitude,Longitude


In [178]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    

In [179]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [180]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Use geopy library to get the coordinates of Manhattan

In [208]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7896239, -73.9598939.


#### Use Foursquare API to search for Yoga Studios in Manhattan

In [209]:
search_query = 'Yoga'
radius = 5000
print(search_query + ' .... OK!')

Yoga .... OK!


In [210]:
url = 'http://random.labs.behindthedot.com/foursquare-cache/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'http://random.labs.behindthedot.com/foursquare-cache/v2/venues/search?client_id=EFE5MSWQNT0Y3F5EEU2ZB2YWSYINQQMTF5ITISJ34PI2OOU4&client_secret=CXOODZSKSF0JETLUQ43ZSXEZ1TAXMSAB0S424NPOWOPWVF4X&ll=40.7896239,-73.9598939&v=20191008&query=Yoga&radius=5000&limit=100'

In [214]:
results = requests.get(url).json()

In [215]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a049afcf964a5200f721fe3,203 E 86th St,...,"[203 E 86th St (at 3rd Ave), New York, NY 1002...","[{'label': 'display', 'lat': 40.77872347864471...",40.778723,-73.953772,NaN,10028,NY,Pure Yoga East,v-1570600480,NaN
1,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b365ac3f964a520773325e3,204 W 77th St,...,"[204 W 77th St (at Amsterdam Ave), New York, N...","[{'label': 'display', 'lat': 40.7816589375539,...",40.781659,-73.979545,NaN,10024,NY,Pure Yoga West,v-1570600480,NaN
2,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",344817,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/motorino-upper-w...,False,516827db72da218d58fdf80b,75 W 85th St Apt B,...,[75 W 85th St Apt B (btwn Columbus Ave. & Cent...,"[{'label': 'display', 'lat': 40.78568220087445...",40.785682,-73.972310,NaN,10024,NY,Upper West Side Yoga and Wellness,v-1570600480,53981514
3,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b55d36ff964a520b3f127e3,140 West 83rd Street - Garden Level,...,[140 West 83rd Street - Garden Level (Amsterda...,"[{'label': 'display', 'lat': 40.78520461497391...",40.785205,-73.975170,NaN,10024,NY,NY Loves Yoga,v-1570600480,50190089
4,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ad7bdb0f964a5204d0e21e3,132 E 85th St,...,"[132 E 85th St (Lexington Ave), New York, NY 1...","[{'label': 'display', 'lat': 40.77890950875221...",40.778910,-73.956160,NaN,10028,NY,New York Yoga - Hot Yoga,v-1570600480,48664255


In [216]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Pure Yoga East,Yoga Studio,203 E 86th St,US,New York,United States,at 3rd Ave,1318,"[203 E 86th St (at 3rd Ave), New York, NY 1002...","[{'label': 'display', 'lat': 40.77872347864471...",40.778723,-73.953772,NaN,10028,NY,4a049afcf964a5200f721fe3
1,Pure Yoga West,Yoga Studio,204 W 77th St,US,New York,United States,at Amsterdam Ave,1878,"[204 W 77th St (at Amsterdam Ave), New York, N...","[{'label': 'display', 'lat': 40.7816589375539,...",40.781659,-73.979545,NaN,10024,NY,4b365ac3f964a520773325e3
2,Upper West Side Yoga and Wellness,Yoga Studio,75 W 85th St Apt B,US,New York,United States,btwn Columbus Ave. & Central Park West,1134,[75 W 85th St Apt B (btwn Columbus Ave. & Cent...,"[{'label': 'display', 'lat': 40.78568220087445...",40.785682,-73.972310,NaN,10024,NY,516827db72da218d58fdf80b
3,NY Loves Yoga,Yoga Studio,140 West 83rd Street - Garden Level,US,New York,United States,Amsterdam and Columbus Ave,1378,[140 West 83rd Street - Garden Level (Amsterda...,"[{'label': 'display', 'lat': 40.78520461497391...",40.785205,-73.975170,NaN,10024,NY,4b55d36ff964a520b3f127e3
4,New York Yoga - Hot Yoga,Yoga Studio,132 E 85th St,US,New York,United States,Lexington Ave,1233,"[132 E 85th St (Lexington Ave), New York, NY 1...","[{'label': 'display', 'lat': 40.77890950875221...",40.778910,-73.956160,NaN,10028,NY,4ad7bdb0f964a5204d0e21e3
5,Life in Motion: Yoga Cycling Barre,Yoga Studio,2744 Broadway,US,New York,United States,@ 105th Street,1387,"[2744 Broadway (@ 105th Street), New York, NY ...","[{'label': 'display', 'lat': 40.80051468505158...",40.800515,-73.967910,NaN,10025,NY,4a86b49cf964a5201d0120e3
6,Spring Into Yoga- Central Park,Yoga Studio,102nd St and Fifth Avenue,US,New York,United States,"102nd st, Fifth Ave",595,"[102nd St and Fifth Avenue (102nd st, Fifth Av...","[{'label': 'display', 'lat': 40.79225934919049...",40.792259,-73.953744,NaN,10029,NY,51bd486babd8c63049739fd8
7,Harlem Yoga Studio,Yoga Studio,44 W 125th St Fl 3,US,New York,United States,btw Lenox and 5th,2359,"[44 W 125th St Fl 3 (btw Lenox and 5th), New Y...","[{'label': 'display', 'lat': 40.80707697816692...",40.807077,-73.944010,NaN,10027,NY,4d48fce9a0ef548126b5edf6
8,Yoga to the People,Yoga Studio,1017 Avenue of the Americas,US,New York,United States,38th St,4695,"[1017 Avenue of the Americas (38th St), New Yo...","[{'label': 'display', 'lat': 40.7523979170956,...",40.752398,-73.986091,NaN,10018,NY,4b4ce47cf964a5203bc326e3
9,Yoga Caliente/Soapstone Gallery,Coworking Space,2035 2nd Ave,US,New York,United States,NaN,1401,"[2035 2nd Ave, New York, NY 10029, United States]","[{'label': 'display', 'lat': 40.7897786, 'lng'...",40.789779,-73.943262,NaN,10029,NY,4df80b3b62847bdf4feb4920


#### Run Folium map to show Yoga Studios in Manhattan

In [218]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add the Yoga Studios as red circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### Narrow down search by Neighbourhood

In [219]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [220]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [221]:
address = 'Marble Hill, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Marble Hill are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Marble Hill are 40.8762983, -73.9104292.


In [222]:
search_query = 'Yoga'
radius = 1000
limit = 25
print(search_query + ' .... OK!')

Yoga .... OK!


In [223]:
url = 'http://random.labs.behindthedot.com/foursquare-cache/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'http://random.labs.behindthedot.com/foursquare-cache/v2/venues/search?client_id=EFE5MSWQNT0Y3F5EEU2ZB2YWSYINQQMTF5ITISJ34PI2OOU4&client_secret=CXOODZSKSF0JETLUQ43ZSXEZ1TAXMSAB0S424NPOWOPWVF4X&ll=40.8762983,-73.9104292&v=20191008&query=Yoga&radius=1000&limit=100'

In [224]:
results = requests.get(url).json()

results

{'meta': {'code': 200, 'requestId': '5d9d74e9dd0f85001be975e7'},
 'response': {'venues': [{'id': '4baf59e8f964a520a6f93be3',
    'name': 'Bikram Yoga',
    'location': {'address': '5500 Broadway',
     'crossStreet': '230th Street',
     'lat': 40.876843690797934,
     'lng': -73.90620384419528,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.876843690797934,
       'lng': -73.90620384419528}],
     'distance': 360,
     'postalCode': '10463',
     'cc': 'US',
     'city': 'Bronx',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['5500 Broadway (230th Street)',
      'Bronx, NY 10463',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d102941735',
      'name': 'Yoga Studio',
      'pluralName': 'Yoga Studios',
      'shortName': 'Yoga Studio',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1570600216',
    'hasPer

In [225]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,4baf59e8f964a520a6f93be3,5500 Broadway,US,Bronx,United States,230th Street,360,"[5500 Broadway (230th Street), Bronx, NY 10463...","[{'label': 'display', 'lat': 40.87684369079793...",40.876844,-73.906204,10463,NY,Bikram Yoga,v-1570600216
1,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,4af09262f964a5204edd21e3,5000 Broadway,US,New York,United States,Btwn 212th St & 213 St,1102,"[5000 Broadway (Btwn 212th St & 213 St), New Y...","[{'label': 'display', 'lat': 40.8682294804136,...",40.868229,-73.918028,10034,NY,Bread and Yoga,v-1570600216
2,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,589f4a9901f0772d80390090,3264 Johnson Ave,US,Bronx,United States,West 235th,471,"[3264 Johnson Ave (West 235th), Bronx, NY 1046...","[{'label': 'display', 'lat': 40.880527, 'lng':...",40.880527,-73.910067,10463,NY,One Yoga For All,v-1570600216
3,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,5949b85f829b0c4ac0d3abc1,Inwood Hill Park,US,New York,United States,NaN,858,"[Inwood Hill Park, New York, NY 10034, United ...","[{'label': 'display', 'lat': 40.873102, 'lng':...",40.873102,-73.919715,10034,NY,Summer Season Yoga,v-1570600216
4,"[{'id': '4bf58dd8d48988d101941735', 'name': 'M...",False,513ba5b9e4b09b144de4210d,"5762 Broadway, Bronx 10463",US,Bronx,United States,NaN,1118,"[5762 Broadway, Bronx 10463, Bronx, NY, United...","[{'label': 'display', 'lat': 40.88379203459271...",40.883792,-73.901573,NaN,NY,"East Coast United Brazilian Jiu Jitsu , MMA & ...",v-1570600216


In [226]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Bikram Yoga,Yoga Studio,5500 Broadway,US,Bronx,United States,230th Street,360,"[5500 Broadway (230th Street), Bronx, NY 10463...","[{'label': 'display', 'lat': 40.87684369079793...",40.876844,-73.906204,10463,NY,4baf59e8f964a520a6f93be3
1,Bread and Yoga,Yoga Studio,5000 Broadway,US,New York,United States,Btwn 212th St & 213 St,1102,"[5000 Broadway (Btwn 212th St & 213 St), New Y...","[{'label': 'display', 'lat': 40.8682294804136,...",40.868229,-73.918028,10034,NY,4af09262f964a5204edd21e3
2,One Yoga For All,Yoga Studio,3264 Johnson Ave,US,Bronx,United States,West 235th,471,"[3264 Johnson Ave (West 235th), Bronx, NY 1046...","[{'label': 'display', 'lat': 40.880527, 'lng':...",40.880527,-73.910067,10463,NY,589f4a9901f0772d80390090
3,Summer Season Yoga,Yoga Studio,Inwood Hill Park,US,New York,United States,NaN,858,"[Inwood Hill Park, New York, NY 10034, United ...","[{'label': 'display', 'lat': 40.873102, 'lng':...",40.873102,-73.919715,10034,NY,5949b85f829b0c4ac0d3abc1
4,"East Coast United Brazilian Jiu Jitsu , MMA & ...",Martial Arts Dojo,"5762 Broadway, Bronx 10463",US,Bronx,United States,NaN,1118,"[5762 Broadway, Bronx 10463, Bronx, NY, United...","[{'label': 'display', 'lat': 40.88379203459271...",40.883792,-73.901573,NaN,NY,513ba5b9e4b09b144de4210d


In [227]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14)

# add the Yoga Studios as red circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### Check rating of Yoga Studio

In [230]:
venue_id = '4baf59e8f964a520a6f93be3' # ID of Bikram Yoga
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()

In [231]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.9


### Investigate Nearby Areas

In [88]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [137]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [135]:
url = 'http://random.labs.behindthedot.com/foursquare-cache/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'http://random.labs.behindthedot.com/foursquare-cache/v2/venues/search?client_id=3HFJKTUWXGIUOIDQ534I001CU0LI1EF13AGFLUQ1AJ21HVUP&client_secret=XHJOMUPLKWQN0KKSIQUVTB53HF2DUNDWMD3Q0EW4Q5FKGU54&ll=43.653963,-79.387207&v=20191008&query=Tennis&radius=1500&limit=25'

In [139]:
radius = 500

LIMIT = 25

url = 'http://random.labs.behindthedot.com/foursquare-cache/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

results

{'meta': {'code': 200, 'requestId': '5d9d3b6ba30619003976ad55'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

In [140]:
url

'http://random.labs.behindthedot.com/foursquare-cache/v2/venues/explore?&client_id=3HFJKTUWXGIUOIDQ534I001CU0LI1EF13AGFLUQ1AJ21HVUP&client_secret=XHJOMUPLKWQN0KKSIQUVTB53HF2DUNDWMD3Q0EW4Q5FKGU54&v=20191008&ll=40.87655077879964,-73.91065965862981&radius=500&limit=25'

In [103]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [104]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


In [26]:
nearby_venues.head(10)

,name,categories,lat,lng
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204
1,Arturo's,Pizza Place,40.874412,-73.910271
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Sam's Pizza,Pizza Place,40.879435,-73.905859
4,Starbucks,Coffee Shop,40.877531,-73.905582
5,Estrellita Poblana V,Mexican Restaurant,40.879687,-73.906257
6,Loeser's Delicatessen,Sandwich Place,40.879242,-73.905471
7,Blink Fitness Riverdale,Gym,40.877147,-73.905837
8,El Malecon,Caribbean Restaurant,40.879338,-73.904457
9,Dunkin',Donut Shop,40.877136,-73.906666


In [61]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 25
    
    RADIUS = 500
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'http://random.labs.behindthedot.com/foursquare-cache/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
url

'http://random.labs.behindthedot.com/foursquare-cache/v2/venues/explore?&client_id=3HFJKTUWXGIUOIDQ534I001CU0LI1EF13AGFLUQ1AJ21HVUP&client_secret=XHJOMUPLKWQN0KKSIQUVTB53HF2DUNDWMD3Q0EW4Q5FKGU54&v=20191008&ll=40.74561857141855,-73.75494976234332&radius=1000&limit=25'

In [65]:
manhattan_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [32]:
print(manhattan_venues.shape)
manhattan_venues.head()

(4825, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station


In [66]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,25,25,25,25,25,25
Annadale,9,9,9,9,9,9
Arden Heights,4,4,4,4,4,4
Arlington,7,7,7,7,7,7
Arrochar,19,19,19,19,19,19
Arverne,19,19,19,19,19,19
Astoria,25,25,25,25,25,25
Astoria Heights,14,14,14,14,14,14
Auburndale,21,21,21,21,21,21


In [67]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 362 uniques categories.


#### Prepare data for Clustering

In [89]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,...,Vietnamese Restaurant,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
manhattan_onehot.shape

(5582, 362)

In [91]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()


In [232]:
manhattan_grouped_Yoga = manhattan_grouped.sort_values('Yoga Studio',ascending=False)

manhattan_grouped_Yoga.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,...,Vietnamese Restaurant,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
124,Hamilton Heights,0.08,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.04,0.00,0.0,0.0
195,North Side,0.08,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
35,Brooklyn Heights,0.08,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.04,0.0,0.0,0.0,0.0,0.0,0.00,0.04,0.0,0.0
27,Boerum Hill,0.08,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
197,Oakland Gardens,0.08,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.04,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0


In [45]:
Oakland = neighborhoods['Neighborhood']=='Oakland Gardens'

Oakland2 = neighborhoods[Oakland]

Oakland2.head()

,Borough,Neighborhood,Latitude,Longitude
161,Queens,Oakland Gardens,40.745619,-73.75495


In [46]:
neighborhood_latitude = neighborhoods.loc[161, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[161, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[161, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Oakland Gardens are 40.74561857141855, -73.75494976234332.


In [200]:
radius = 500
LIMIT = 10

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [201]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [202]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [203]:
nearby_venues

,name,categories,lat,lng
0,Gino's Pizzeria,Pizza Place,40.748445,-73.756498
1,Bi Won Korean Restaurant,Korean Restaurant,40.747491,-73.755944
2,Flex Hot Yoga,Yoga Studio,40.749555,-73.755024
3,Fukuoka Shabu Shabu,Taiwanese Restaurant,40.748990,-73.756717
4,Slim's Bagels & Bialy's,Bagel Shop,40.749271,-73.755754
5,FlexHotYoga,Yoga Studio,40.749082,-73.754943
6,Chilsung Garden,Korean Restaurant,40.749320,-73.755368
7,Rolly Kimbab,Korean Restaurant,40.748531,-73.756335
8,Little Dumpling,Chinese Restaurant,40.749046,-73.756210
9,Starbucks,Coffee Shop,40.749086,-73.756312


In [253]:
num_top_venues = 10

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                 venue  freq
0          Pizza Place  0.16
1          Supermarket  0.12
2             Pharmacy  0.08
3   Chinese Restaurant  0.08
4  Fried Chicken Joint  0.04
5       Breakfast Spot  0.04
6          Gas Station  0.04
7   Spanish Restaurant  0.04
8          Bus Station  0.04
9                  Spa  0.04
----Annadale----
            venue  freq
0    Dance Studio  0.11
1  Cosmetics Shop  0.11
2   Train Station  0.11
3       Pet Store  0.11
4        Pharmacy  0.11
5      Restaurant  0.11
6     Pizza Place  0.11
7      Sports Bar  0.11
8           Diner  0.11
9           Plaza  0.00
----Arden Heights----
                   venue  freq
0            Coffee Shop  0.25
1               Bus Stop  0.25
2            Pizza Place  0.25
3               Pharmacy  0.25
4            Yoga Studio  0.00
5            Opera House  0.00
6  Outdoors & Recreation  0.00
7      Outdoor Sculpture  0.00
8      Other Repair Shop  0.00
9        Other Nightlife  0.00
----Arlington----
  

In [73]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### K-means Clustering

In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Chinese Restaurant,Pharmacy,Fried Chicken Joint,Martial Arts Dojo,Fast Food Restaurant,Electronics Store,Gas Station,Bus Station
1,Annadale,Restaurant,Pizza Place,Pet Store,Pharmacy,Diner,Train Station,Sports Bar,Cosmetics Shop,Dance Studio,Eye Doctor
2,Arden Heights,Bus Stop,Pizza Place,Coffee Shop,Pharmacy,Field,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant
3,Arlington,Bus Stop,Boat or Ferry,Deli / Bodega,Intersection,Grocery Store,American Restaurant,Fish & Chips Shop,Factory,Falafel Restaurant,Farm
4,Arrochar,Bus Stop,Deli / Bodega,Italian Restaurant,Athletics & Sports,Middle Eastern Restaurant,Supermarket,Mediterranean Restaurant,Liquor Store,Sandwich Place,Outdoors & Recreation


In [75]:
# set number of clusters
kclusters = 6

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 5, 5, 4, 4, 1, 4, 4], dtype=int32)

In [95]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Coffee Shop,Yoga Studio,Gym,Tennis Stadium,Supplement Shop,Steakhouse,Miscellaneous Shop,Shopping Mall,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,4,Chinese Restaurant,Noodle House,Sandwich Place,Pizza Place,Bakery,Spa,Museum,New American Restaurant,Garden Center,Bike Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Wine Shop,Park,Café,Bakery,Pool,New American Restaurant,Market,Frozen Yogurt Shop,Breakfast Spot,Burger Joint
3,Manhattan,Inwood,40.867684,-73.921210,4,Café,Wine Bar,Bakery,Park,Mexican Restaurant,Yoga Studio,Bistro,Farmers Market,Latin American Restaurant,Frozen Yogurt Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Yoga Studio,Café,Cocktail Bar,Coffee Shop,Caribbean Restaurant,Mexican Restaurant,Indian Restaurant,Historic Site,Bakery,Mediterranean Restaurant


In [233]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [98]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [99]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [59]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 5, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [57]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [100]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Coffee Shop,Yoga Studio,Gym,Tennis Stadium,Supplement Shop,Steakhouse,Miscellaneous Shop,Shopping Mall,Seafood Restaurant
1,Chinatown,Chinese Restaurant,Noodle House,Sandwich Place,Pizza Place,Bakery,Spa,Museum,New American Restaurant,Garden Center,Bike Shop
2,Washington Heights,Wine Shop,Park,Café,Bakery,Pool,New American Restaurant,Market,Frozen Yogurt Shop,Breakfast Spot,Burger Joint
3,Inwood,Café,Wine Bar,Bakery,Park,Mexican Restaurant,Yoga Studio,Bistro,Farmers Market,Latin American Restaurant,Frozen Yogurt Shop
4,Hamilton Heights,Yoga Studio,Café,Cocktail Bar,Coffee Shop,Caribbean Restaurant,Mexican Restaurant,Indian Restaurant,Historic Site,Bakery,Mediterranean Restaurant
5,Manhattanville,Italian Restaurant,Mexican Restaurant,Coffee Shop,Ramen Restaurant,Seafood Restaurant,Other Nightlife,Juice Bar,BBQ Joint,Dumpling Restaurant,Bike Trail
6,Central Harlem,French Restaurant,African Restaurant,American Restaurant,Dessert Shop,Bagel Shop,Spa,Fried Chicken Joint,Café,Music Venue,Caribbean Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Dance Studio,Sandwich Place,Beer Bar,Thai Restaurant,Café,Gym,Latin American Restaurant,Cocktail Bar
8,Upper East Side,Hotel,Gym / Fitness Center,Boutique,Bar,Seafood Restaurant,Bakery,Coffee Shop,Park,Chocolate Shop,Shoe Store
9,Yorkville,Italian Restaurant,Wine Shop,Deli / Bodega,Coffee Shop,Gym / Fitness Center,Sandwich Place,Sushi Restaurant,Gym,Thai Restaurant,Monument / Landmark


### Toronto Data

#### Import data from Wikipedia

In [37]:
postalcodes_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [38]:
postalcodes_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


#### Clean Data

In [40]:
postalcodes_df=postalcodes_df[postalcodes_df.Borough != 'Not assigned']

postalcodes_df['Neighbourhood']=postalcodes_df['Neighbourhood'].replace('Not assigned', postalcodes_df['Borough'])

postalcodes_df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [41]:
pc2_df=postalcodes_df.groupby(['Postcode','Borough'],as_index=False, sort=False).agg(','.join)
pc2_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [42]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(pc2_df['Borough'].unique()),
        pc2_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighbourhoods.


#### Import Coordinates

In [43]:
coords_df = pd.read_csv("http://cocl.us/Geospatial_data")

coords_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
pc2_df.rename(columns={'Postcode': 'Postal Code'}, inplace=True)
pc2_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


#### Merge Data

In [45]:
df_final = pd.merge(pc2_df, coords_df, on='Postal Code')

df_final.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### Search for Tennis Courts in Toronto

In [236]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [237]:
search_query = 'Tennis'
radius = 1500
print(search_query + ' .... OK!')

Tennis .... OK!


In [238]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=EFE5MSWQNT0Y3F5EEU2ZB2YWSYINQQMTF5ITISJ34PI2OOU4&client_secret=CXOODZSKSF0JETLUQ43ZSXEZ1TAXMSAB0S424NPOWOPWVF4X&ll=43.653963,-79.387207&v=20191008&query=Tennis&radius=1500&limit=100'

In [239]:
results = requests.get(url).json()

In [240]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

In [241]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Moss Park Tennis Courts,Tennis Court,Moss Park,CA,Toronto,Canada,NaN,1240,"[Moss Park, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.6553367165992,...",43.655337,-79.371923,NaN,ON,4c54aeb330f92d7f73b8daba
1,Tennis Court,Tennis Court,Pinnacle Centre,CA,Toronto,Canada,Yonge & Harbour St.,1462,"[Pinnacle Centre (Yonge & Harbour St.), Toront...","[{'label': 'display', 'lat': 43.64340861975214...",43.643409,-79.376392,M5E 2A1,ON,4e4ac9172271ac3f6bd93d81
2,Tennis Courts at Skymark Condos,Tennis Court,Skymark Condos,CA,Toronto,Canada,NaN,497,"[Skymark Condos, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65107794416522...",43.651078,-79.382486,NaN,ON,4c459b2274ffc928db1c77cc
3,Viewmont tennis courts,Tennis Court,Viewmount park,CA,Toronto,Canada,NaN,396,"[Viewmount park, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.6504, 'lng': -...",43.650400,-79.386996,NaN,ON,4dc885a2d22d4e36b4494240
4,tennis court,General Entertainment,16 Yonge St,CA,Toronto,Canada,NaN,1519,"[16 Yonge St, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64319917172176...",43.643199,-79.375602,NaN,ON,5105bd45e4b0e721c999af47


In [243]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add the Tennis Courts as red circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### Search for Yoga Studios in Toronto

In [244]:
search_query = 'Yoga'
radius = 1500
print(search_query + ' .... OK!')

Yoga .... OK!


In [245]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=EFE5MSWQNT0Y3F5EEU2ZB2YWSYINQQMTF5ITISJ34PI2OOU4&client_secret=CXOODZSKSF0JETLUQ43ZSXEZ1TAXMSAB0S424NPOWOPWVF4X&ll=43.653963,-79.387207&v=20191008&query=Yoga&radius=1500&limit=100'

In [246]:
results = requests.get(url).json()

In [247]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

In [248]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Yoga Sanctuary,Yoga Studio,"2 College St. W, Suite 306",CA,Toronto,Canada,at Yonge St.,886,"[2 College St. W, Suite 306 (at Yonge St.), To...","[{'label': 'display', 'lat': 43.66149949759759...",43.661499,-79.383636,M5G 1K3,ON,4aeb5989f964a5203ec121e3
1,Yoga Tree Downtown,Yoga Studio,140 Spadina Ave Unit A201,CA,Toronto,Canada,at Richmond St,983,"[140 Spadina Ave Unit A201 (at Richmond St), T...","[{'label': 'display', 'lat': 43.64790246011444...",43.647902,-79.396086,M5V 2L4,ON,4d90ca3cc1b1721e93666346
2,Core Studio Yoga & Pilates,Yoga Studio,553 Queen St. W,CA,Toronto,Canada,at Augusta Ave.,1243,"[553 Queen St. W (at Augusta Ave.), Toronto ON...","[{'label': 'display', 'lat': 43.64791967343419...",43.647920,-79.400196,M5V 2B6,ON,4adf8284f964a520477b21e3
3,Tula Yoga Studio,Yoga Studio,"352 Adelaide St W., 3rd Floor",CA,Toronto,Canada,NaN,882,"[352 Adelaide St W., 3rd Floor, Toronto ON M5V...","[{'label': 'display', 'lat': 43.64746372311456...",43.647464,-79.393472,M5V 1R8,ON,53dea941498efed38c292d40
4,Aum Centre Yoga,Yoga Studio,NaN,CA,Toronto,Canada,NaN,1089,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.657688, 'lng':...",43.657688,-79.399723,NaN,ON,5081d1e4e4b0921f5bb47c6f
5,Yoga Tree Bay and Dundas,Yoga Studio,123 Dundas Street West,CA,Toronto,Canada,NaN,282,"[123 Dundas Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65556089904130...",43.655561,-79.384478,NaN,ON,542044a3498e78ca08c471bd
6,889 Yoga,Yoga Studio,550 Wellington Street West,CA,Toronto,Canada,Bathurst,1701,"[550 Wellington Street West (Bathurst), Toront...","[{'label': 'display', 'lat': 43.64283283704247...",43.642833,-79.401683,M5V 2V4,ON,4eb340f1e5e8743704d75e4a
7,Yoga Tree,None,NaN,CA,NaN,Canada,NaN,286,[Canada],"[{'label': 'display', 'lat': 43.655697, 'lng':...",43.655697,-79.384577,NaN,NaN,544e5ddc498e91fa4e1edb22
8,Yoga Studio,Yoga Studio,"100 King St. West, Suite 5600",CA,Toronto,Canada,Bay St.,301,"[100 King St. West, Suite 5600 (Bay St.), Toro...","[{'label': 'display', 'lat': 43.65322456230742...",43.653225,-79.383603,NaN,ON,4d80128e86efba7a378e2e0d
9,Iam Yoga,Yoga Studio,NaN,CA,Toronto,Canada,NaN,911,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.645858, 'lng':...",43.645858,-79.388854,NaN,ON,5898c9350a3d54567ed9120a


In [251]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map